In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]=""

import numpy as np
import tensorflow as tf

import gensim
import keras as k

from keras.preprocessing.text import text_to_word_sequence, one_hot, Tokenizer
from keras.preprocessing.sequence import pad_sequences

from keras.models import Model, load_model
from keras.layers import Input, LSTM, Dense
from keras.callbacks import ModelCheckpoint, CSVLogger

import util as u

paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress
Using TensorFlow backend.


In [2]:
# config = tf.ConfigProto(device_count={"CPU": 8})
# k.backend.tensorflow_backend.set_session(tf.Session(config=config))

### Initialize Slacker Object

In [3]:
slack = u.initiate_slacker()

Connected to anneke@iitml.


### Load Google's pretrained word2vec

In [4]:
w2v_model = gensim.models.KeyedVectors.load_word2vec_format('../../data/GoogleNews-vectors-negative300.bin', binary=True)

### Load data

In [5]:
X_train_sequence = u.open_pickle('../../data/imdb/X_tr_sample_original.pkl')
X_test_sequence = u.open_pickle('../../data/imdb/X_te_sample_original.pkl')
y_train_target = u.open_pickle('../../data/imdb/y_tr_target_original.pkl')
y_test_target = u.open_pickle('../../data/imdb/y_te_target_original.pkl')

In [6]:
y_train_target = [' '.join(['<UNK>', y]) for y in y_train_target]
y_test_target = [' '.join(['<UNK>', y]) for y in y_test_target]

Initialize constant here

In [7]:
MAX_ENCODER_SEQ_LEN = 81
MAX_DECODER_SEQ_LEN = 6 #include <UNK>
EMBEDDING_DIM = 300
LATENT_DIM = 100

In [8]:
# Tokenize the train sequence data
tokenizer = k.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(X_train_sequence)

# Generate text to integer sequence with post padding
# X_tr_padded = pad_sequences(tokenizer.texts_to_sequences(X_train_sequence), maxlen=81, padding='post', truncating='post')
# y_tr_padded = pad_sequences(tokenizer.texts_to_sequences(y_train_target), maxlen=5, padding='post', truncating='post')

In [9]:
# X_tr_padded[0]

In [10]:
NUM_DECODER_TOKENS = len(tokenizer.word_index)

In [11]:
index_to_word = {v: k for k, v in tokenizer.word_index.items()}
index_to_word[0] = ' '

In [12]:
NUM_DECODER_TOKENS

42406

Initialize the container for input sequence decoder and encoder

In [12]:
encoder_input_data = np.zeros(
    (len(X_train_sequence), MAX_ENCODER_SEQ_LEN, EMBEDDING_DIM),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(y_train_target), MAX_DECODER_SEQ_LEN, EMBEDDING_DIM),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(y_train_target), MAX_DECODER_SEQ_LEN, len(tokenizer.word_index)),
    dtype='float32')

In [14]:
for i, (input_sequence, target_sequence) in enumerate(zip(X_tr_padded, y_tr_padded)):
    # embed the input sequence
    for t, index in enumerate(input_sequence):
        try:
            encoder_input_data[i, t, :] = w2v_model[index_to_word[index]]
        except KeyError as error:
            pass
    
    # embed the input decoder
    for t, index in enumerate(target_sequence):
        try:
            decoder_input_data[i, t, :] = w2v_model[index_to_word[index]]
        except KeyError as error:
            pass
        
    for t, index in enumerate(target_sequence):
        # not include the first <UNK>
        if t>0:
            decoder_target_data[i, t - 1, index] = 1.

# Model callbacks

In [66]:
# checkpoint = k.callbacks.ModelCheckpoint(save_best_only=True, monitor='val_loss', filepath='./300_word2vec_best_model/weights.{epoch:04d}-{val_loss:.3f}.h5')
# csvlogger = k.callbacks.CSVLogger(filename='word2vec_300_history.log', append=True)

### Train

In [75]:
# encoder_inputs = k.layers.Input(shape=(None, EMBEDDING_DIM))
# encoder = LSTM(LATENT_DIM, return_state=True)
# encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# encoder_states = [state_h, state_c]

In [76]:
# decoder_inputs = k.layers.Input(shape=(None, EMBEDDING_DIM))
# decoder_lstm = LSTM(LATENT_DIM, return_sequences=True, return_state=True)
# decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
#                                     initial_state=encoder_states)

# decoder_dense = k.layers.Dense(NUM_DECODER_TOKENS, activation='softmax')
# decoder_outputs = decoder_dense(decoder_outputs)

In [77]:
# train_model = k.models.Model([encoder_inputs, decoder_inputs], decoder_outputs)
# train_model.compile(optimizer='adam', loss='categorical_crossentropy')

In [78]:
# epochs=500
# batch_size=256

In [79]:
# train_model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
#           batch_size=batch_size,
#           epochs=epochs,
#           validation_split=(1./3),
#           verbose=1, callbacks=[checkpoint, csvlogger])

Train on 15168 samples, validate on 7584 samples
Epoch 1/500
15168/15168 [==============================] - 402s 27ms/step - loss: 5.3539 - val_loss: 3.9570


/home/anneke/.local/lib/python3.6/site-packages/keras/engine/network.py:877: UserWarning: Layer lstm_4 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_3/while/Exit_2:0' shape=(?, 100) dtype=float32>, <tf.Tensor 'lstm_3/while/Exit_3:0' shape=(?, 100) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '


Epoch 2/500
15168/15168 [==============================] - 400s 26ms/step - loss: 3.8193 - val_loss: 3.8857
Epoch 3/500
15168/15168 [==============================] - 399s 26ms/step - loss: 3.7246 - val_loss: 3.8293
Epoch 4/500
15168/15168 [==============================] - 399s 26ms/step - loss: 3.6572 - val_loss: 3.7967
Epoch 5/500
15168/15168 [==============================] - 399s 26ms/step - loss: 3.6118 - val_loss: 3.7712
Epoch 6/500
15168/15168 [==============================] - 400s 26ms/step - loss: 3.5796 - val_loss: 3.7558
Epoch 7/500
15168/15168 [==============================] - 400s 26ms/step - loss: 3.5540 - val_loss: 3.7440
Epoch 8/500
15168/15168 [==============================] - 399s 26ms/step - loss: 3.5331 - val_loss: 3.7308
Epoch 9/500
15168/15168 [==============================] - 399s 26ms/step - loss: 3.5038 - val_loss: 3.6959
Epoch 10/500
15168/15168 [==============================] - 399s 26ms/step - loss: 3.4751 - val_loss: 3.6753
Epoch 11/500
15168/15168 [=

Epoch 152/500
15168/15168 [==============================] - 400s 26ms/step - loss: 1.7207 - val_loss: 2.9379
Epoch 153/500
15168/15168 [==============================] - 400s 26ms/step - loss: 1.7088 - val_loss: 2.9516
Epoch 154/500
15168/15168 [==============================] - 400s 26ms/step - loss: 1.6982 - val_loss: 2.9524
Epoch 155/500
15168/15168 [==============================] - 400s 26ms/step - loss: 1.6851 - val_loss: 2.9520
Epoch 156/500
15168/15168 [==============================] - 400s 26ms/step - loss: 1.6752 - val_loss: 2.9506
Epoch 157/500
15168/15168 [==============================] - 400s 26ms/step - loss: 1.6683 - val_loss: 3.0108
Epoch 158/500
15168/15168 [==============================] - 400s 26ms/step - loss: 1.7003 - val_loss: 2.9559
Epoch 159/500
15168/15168 [==============================] - 400s 26ms/step - loss: 1.6585 - val_loss: 2.9586
Epoch 160/500
15168/15168 [==============================] - 400s 26ms/step - loss: 1.6462 - val_loss: 2.9666
Epoch 161/

15168/15168 [==============================] - 401s 26ms/step - loss: 0.8405 - val_loss: 3.5369
Epoch 301/500
15168/15168 [==============================] - 401s 26ms/step - loss: 0.8387 - val_loss: 3.5446
Epoch 302/500
15168/15168 [==============================] - 400s 26ms/step - loss: 0.8328 - val_loss: 3.5562
Epoch 303/500
15168/15168 [==============================] - 401s 26ms/step - loss: 0.8198 - val_loss: 3.5573
Epoch 304/500
15168/15168 [==============================] - 401s 26ms/step - loss: 0.8234 - val_loss: 3.5505
Epoch 305/500
15168/15168 [==============================] - 401s 26ms/step - loss: 0.8208 - val_loss: 3.5702
Epoch 306/500
15168/15168 [==============================] - 401s 26ms/step - loss: 0.8167 - val_loss: 3.5664
Epoch 307/500
15168/15168 [==============================] - 400s 26ms/step - loss: 0.8105 - val_loss: 3.5700
Epoch 308/500
15168/15168 [==============================] - 401s 26ms/step - loss: 0.8046 - val_loss: 3.5800
Epoch 309/500
15168/1516

15168/15168 [==============================] - 401s 26ms/step - loss: 0.4358 - val_loss: 4.2040
Epoch 449/500
15168/15168 [==============================] - 401s 26ms/step - loss: 0.4355 - val_loss: 4.1973
Epoch 450/500
15168/15168 [==============================] - 400s 26ms/step - loss: 0.4346 - val_loss: 4.2152
Epoch 451/500
15168/15168 [==============================] - 401s 26ms/step - loss: 0.4312 - val_loss: 4.1941
Epoch 452/500
15168/15168 [==============================] - 401s 26ms/step - loss: 0.4226 - val_loss: 4.2065
Epoch 453/500
15168/15168 [==============================] - 401s 26ms/step - loss: 0.4380 - val_loss: 4.2252
Epoch 454/500
15168/15168 [==============================] - 401s 26ms/step - loss: 0.4281 - val_loss: 4.2134
Epoch 455/500
15168/15168 [==============================] - 401s 26ms/step - loss: 0.4293 - val_loss: 4.2201
Epoch 456/500
15168/15168 [==============================] - 401s 26ms/step - loss: 0.5775 - val_loss: 4.2080
Epoch 457/500
15168/1516

### Inference on Train/test set

In [13]:
model = load_model('./300_word2vec_best_model/weights.0121-2.897.h5')

encoder_inputs = model.input[0]
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output
encoder_states = [state_h_enc, state_c_enc]
encoder_model = Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]
decoder_state_input_h = Input(shape=(LATENT_DIM,), name='input_5')
decoder_state_input_c = Input(shape=(LATENT_DIM,), name='input_6')

decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_lstm = model.layers[3]

decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
decoder_inputs, initial_state=decoder_states_inputs)

decoder_states = [state_h_dec, state_c_dec]

decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
                    [decoder_inputs] + decoder_states_inputs,
                    [decoder_outputs] + decoder_states
                    )



### Data

In [19]:
max_encoder_seq_length = 81
max_decoder_seq_length = 6

X_te_padded = pad_sequences(tokenizer.texts_to_sequences(X_test_sequence), maxlen=81, padding='post', truncating='post')
y_te_padded = pad_sequences(tokenizer.texts_to_sequences(y_test_target), maxlen=6, padding='post', truncating='post')

test_encoder_input_data = np.zeros(
    (len(X_test_sequence), max_encoder_seq_length, EMBEDDING_DIM),
    dtype='float32')
test_decoder_input_data = np.zeros(
    (len(y_test_target), max_decoder_seq_length, EMBEDDING_DIM),
    dtype='float32')
test_decoder_target_data = np.zeros(
    (len(y_test_target), max_decoder_seq_length, len(tokenizer.word_index)),
    dtype='float32')

test_sequence = []
test_target_sequence = []

for sample in X_test_sequence:
    test_sequence.append(sample.split())
for target in y_test_target:
    test_target_sequence.append(target.split())
    
# 100-dim -> input sequence, input decoder
# 42K-dim -> output sequence.

for i, (input_text, target_text, target_padded) in enumerate(zip(test_sequence, test_target_sequence, y_te_padded)):
    for t, word in enumerate(input_text):
        try:
            test_encoder_input_data[i, t, :] = w2v_model[word]
        except KeyError as error:
            continue
    
    for t, word in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        try:
            test_decoder_input_data[i, t, :] = w2v_model[word]
        except KeyError as error:
            continue
        
    for t, word in enumerate(target_padded):
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            test_decoder_target_data[i, t - 1, word] = 1.
    

In [20]:
word_index = tokenizer.word_index
reverse_word_index = dict((i,word) for word,i in word_index.items())

In [21]:
def generate_candidate_list(X):
    y_candidate = []
    
    for i in range(X.shape[0]-1-5):
        y_candidate.append(X[i:i+5])
    
    return np.asarray(y_candidate)

def intersection(lst1, lst2): 
    lst3 = [value for value in lst1 if value in lst2] 
    return lst3 

def target_index(doc_idx, candidate_seq, y):
    for i,j in enumerate(candidate_seq):
        if len(intersection(j, y)) == len(y):
            return i
    return -1

# doc num, doc index argmax

def to_sequence(int_sequence):
    decoded = ''
    for i,intnum in enumerate(int_sequence):
        if intnum == 0:
            word = '<PAD>'
        else:
            word = reverse_word_index[intnum]
        
        if i == len(int_sequence):
            decoded += word
        else:
            decoded += word + ' '
    return decoded

def rouge_one(true, candidate, start_index):
    
    if isinstance(true, str) and isinstance(candidate, str):
        true = true.split()
        candidate = candidate.split()
    
    overlap = [value for value in true[start_index:] if value in candidate[start_index:]] 

    
    if len(true[start_index:]) != 0:
        recall = len(overlap)/len(true[start_index:])
    else:
        recall = 0
    
    if len(candidate[start_index:]):
        precision = len(overlap)/len(candidate[start_index:])
    else:
        precision = 0
    
    if (recall+precision) != 0:    
        f1 = 2*((recall*precision)/(recall+precision))
    else:
        f1 = 0
    
    return recall, precision, f1

In [22]:
# Play with candidate

def test_decode_sequence_target(candidate_states_value, candidate_target_seq):
#     candidate_states_value = encoder_model.predict(input_seq)

    from_candidate_target_seq = np.zeros((1,1, EMBEDDING_DIM))
    
    candidate_token_index = candidate_target_seq[0,0]
    try:
        from_candidate_target_seq[0,0,:] = w2v_model[index_to_word[candidate_token_index]]
    except KeyError as error:
        pass
    
    candidate_joint_log_prob = 0
    
    for i in range(1,5):
        from_candidate_output_tokens, h_true, c_true = decoder_model.predict([from_candidate_target_seq] + candidate_states_value)
    
        candidate_target_prob = from_candidate_output_tokens[0,-1, candidate_target_seq[0,i]]
        candidate_joint_log_prob += np.log(candidate_target_prob)
        
        # get the t+1 input
        
        candidate_token_index = candidate_target_seq[0,i]
        from_candidate_target_seq = np.zeros((1,1,EMBEDDING_DIM))
        try:
            from_candidate_target_seq[0,0,:] = w2v_model[index_to_word[candidate_token_index]]
        except KeyError as error:
            pass
        
        
        candidate_states_value = [h_true, c_true]

    return candidate_joint_log_prob, candidate_target_prob

In [23]:
i=0
file = open("candidate_jll_300_word2vec_imdb_test.csv", "w")

In [24]:
for i,doc in enumerate(X_te_padded):
    y_candidate = generate_candidate_list(doc)
    
    candidate_jll_per_doc = []
    input_seq = test_encoder_input_data[i:i+1]
    
    true_target_index = target_index(i, y_candidate, y_te_padded[i])
    
    # Encode
    candidate_states_value = encoder_model.predict(input_seq)
    
    for j in range(y_candidate.shape[0]):
        candidate_seq = y_candidate[j:j+1]
        candidate_jll_slide, candidate_last_prob = test_decode_sequence_target(candidate_states_value, candidate_seq)
        candidate_jll_per_doc.append(candidate_jll_slide)
   
    candidate_jll_per_doc = np.asarray(candidate_jll_per_doc)
    max_jll_index = np.argmax(candidate_jll_per_doc)
    true_target_jll = np.around(candidate_jll_per_doc[true_target_index],5)
    max_candidate_jll = np.around(candidate_jll_per_doc[max_jll_index],5)
    
    [precision, recall, f_score] = rouge_one(y_test_target[i], to_sequence(y_candidate[max_jll_index]), 1)
    
    file.write('%d\t%d\t%s\t%d\t%s\t%d\t%.5f\t%.5f\t%.5f\t%d\t%.5f\t%.5f\t%.5f\t%.5f\t%.5f\n' %(i, true_target_index, y_test_target[i],
                                                            max_jll_index, to_sequence(y_candidate[max_jll_index]),
                                                            -(true_target_index-max_jll_index),
                                                            true_target_jll, max_candidate_jll,
                                                            np.absolute(true_target_jll-max_candidate_jll),
                                                            len(intersection(y_te_padded[i], y_candidate[max_jll_index])),
                                                            np.exp(true_target_jll/4), np.exp(max_candidate_jll/4),
                                                            precision, recall, f_score))
    
    

    
#     print('%d\t%d\t%s\t%d\t%s\t%d\t%.5f\t%.5f\t%.5f\t%d\n' %(i, true_target_index, y['text'][i],
#                                                             max_jll_index, to_sequence(y_candidate[max_jll_index]),
#                                                             -(true_target_index-max_jll_index),
#                                                             true_target_jll, max_candidate_jll,
#                                                             np.absolute(true_target_jll-max_candidate_jll),
#                                                             len(intersection(y['padded'][i], y_candidate[max_jll_index]))))
    if i % 1000 == 0:
#         print('Processing document %d...' %(i))
        msg = 'glove 100 test: processing document ' + str(i)
        u.slack_post_message(slack, msg, 'deep-learning', 'test dataset')
        print(msg)
        
#     i += 1
    
file.close()
# report_stats('Processing DONE', 'deep-learning')

glove 100 test: processing document 0
glove 100 test: processing document 1000
glove 100 test: processing document 2000
glove 100 test: processing document 3000
glove 100 test: processing document 4000
glove 100 test: processing document 5000
glove 100 test: processing document 6000
glove 100 test: processing document 7000
glove 100 test: processing document 8000
glove 100 test: processing document 9000
glove 100 test: processing document 10000
glove 100 test: processing document 11000
glove 100 test: processing document 12000
glove 100 test: processing document 13000
glove 100 test: processing document 14000
glove 100 test: processing document 15000
glove 100 test: processing document 16000
glove 100 test: processing document 17000
glove 100 test: processing document 18000
glove 100 test: processing document 19000
glove 100 test: processing document 20000
glove 100 test: processing document 21000
glove 100 test: processing document 22000


In [ ]:
u.slack_post_message(slack, msg, 'deep-learning', 'test')